In [45]:
import re
import requests
import datetime
import os
# from langchain.agents import tool
from collections import defaultdict
from datetime import datetime, timedelta

# API 호출 test

In [22]:
service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
key = service_key or os.getenv("KMA_SERVICE_KEY")
if not key:
    raise ValueError("service_key가 필요합니다.")

params = {
"serviceKey": key,
"numOfRows": "1000",
"pageNo": "1",
"dataType": "JSON",
"base_date": '20250811',
"base_time": '0500',
"nx": 60,
"ny": 127
}

try:
    response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params).json()
except requests.RequestException as e:
    raise f"HTTP 오류: {e}"

try:
    data = response['response']['body']['items']['item']

except KeyError as e:
    raise KeyError(f"데이터가 없습니다. input 조건을 확인하세요: {e}") from None

data
# 날짜/시간 단위로 category 묶기
grouped = defaultdict(dict)
for item in data:
    key = (item['fcstDate'], item['fcstTime'])
    grouped[key][item['category']] = item['fcstValue']

grouped

defaultdict(dict,
            {('20250811', '0600'): {'TMP': '24',
              'UUU': '-0.7',
              'VVV': '0.1',
              'VEC': '101',
              'WSD': '0.8',
              'SKY': '3',
              'PTY': '0',
              'POP': '20',
              'WAV': '-999',
              'PCP': '강수없음',
              'REH': '90',
              'SNO': '적설없음'},
             ('20250811', '0700'): {'TMP': '25',
              'UUU': '-0.8',
              'VVV': '0.8',
              'VEC': '133',
              'WSD': '1.2',
              'SKY': '3',
              'PTY': '0',
              'POP': '20',
              'WAV': '-999',
              'PCP': '강수없음',
              'REH': '85',
              'SNO': '적설없음'},
             ('20250811', '0800'): {'TMP': '25',
              'UUU': '-0.9',
              'VVV': '1.5',
              'VEC': '146',
              'WSD': '1.8',
              'SKY': '3',
              'PTY': '0',
              'POP': '20',
              'WAV': '-999',


In [23]:
def get_weather_info(base_date, base_time, nx, ny):
    """
    날씨 정보를 가져오는 tool
    반환: {(fcstDate, fcstTime): {category: fcstValue, ...}, ...}
    ex. input 데이터 예시 
    base_date = "20250806", "base_time": "0500", "nx": "55", "ny": "127"
    # base_time = 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 중 1개
    """

    service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
    key = service_key or os.getenv("KMA_SERVICE_KEY")
    if not key:
        raise ValueError("service_key가 필요합니다.")
    
    params = {
    "serviceKey": key,
    "numOfRows": "1000",
    "pageNo": "1",
    "dataType": "JSON",
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
    }
    
    try:
        response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params).json()
    except requests.RequestException as e:
        raise f"HTTP 오류: {e}"

    try:
        data = response['response']['body']['items']['item']
    except KeyError as e:
        raise KeyError(f"데이터가 없습니다. input 조건을 확인하세요: {e}") from None

    # 오늘 날짜 기준 설정
    today = datetime.today()
    
    # 날짜/시간 단위로 category 묶기
    grouped = defaultdict(dict)
    for item in data:
        key = (item['fcstDate'], item['fcstTime'])
        grouped[key][item['category']] = item['fcstValue']

    targets = [
        (1, "13"),  # 1일 후 13시
        (2, "13"),  # 2일 후 13시
        (3, "13")   # 3일 후 13시
    ]


    result = []
    result.append(f"{base_date} {base_time} 단기예보입니다." )

    for days_after, hour in targets:
        target_date = (today + timedelta(days=days_after)).strftime("%Y%m%d")
        target_time = f"{hour}00"

        # 해당 날짜·시간 데이터 가져오기
        categories = grouped.get((target_date, target_time))
        if categories:
            result.append(
                f"{days_after}일 후 {hour}시 기온은 {categories.get('TMP', '-')}℃ 입니다."
            )
        else:
            result.append(
                f"{days_after}일 후 {hour}시 데이터가 없습니다"
            )

    # 보기 좋게 출력
    return print("\n".join(result))

In [24]:
base_date = "20250811"
base_time= "0500" 
nx= "55" 
ny= "127"
get_weather_info(base_date, base_time, nx, ny)

20250811 0500 단기예보입니다.
1일 후 13시 기온은 30℃ 입니다.
2일 후 13시 기온은 26℃ 입니다.
3일 후 13시 데이터가 없습니다


# 도구 정의

In [66]:
# 도구 정의
from langchain_core.tools import tool  # ✅ 여기서 가져오기
@tool("get_word_length", infer_schema=True, return_direct=True)
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool("get_weather_info", infer_schema=True, return_direct=True)
def get_weather_info(base_date: str, base_time: str, nx: int, ny: int) -> str:
    """
    기상청 단기예보 API를 호출해 특정 날짜/시간/좌표의 날씨 정보를 가져옵니다.
    인자는 반드시 JSON 형식으로 작성해야 합니다.
    예시:
    {
        "base_date": "20250811",
        "base_time": "0500",
        "nx": 60,
        "ny": 127
    }
    """

    service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
    key = service_key or os.getenv("KMA_SERVICE_KEY")
    if not key:
        raise ValueError("service_key가 필요합니다.")
    
    params = {
    "serviceKey": key,
    "numOfRows": "1000",
    "pageNo": "1",
    "dataType": "JSON",
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
    }
    
    try:
        response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params).json()
    except requests.RequestException as e:
        raise f"HTTP 오류: {e}"

    try:
        data = response['response']['body']['items']['item']
    except KeyError as e:
        raise KeyError(f"데이터가 없습니다. input 조건을 확인하세요: {e}") from None

    # 오늘 날짜 기준 설정
    today = datetime.today()
    
    # 날짜/시간 단위로 category 묶기
    grouped = defaultdict(dict)
    for item in data:
        key = (item['fcstDate'], item['fcstTime'])
        grouped[key][item['category']] = item['fcstValue']

    targets = [
        (1, "13"),  # 1일 후 13시
        (2, "13"),  # 2일 후 13시
        (3, "13")   # 3일 후 13시
    ]


    result = []
    result.append(f"{base_date} {base_time} 단기예보입니다." )

    for days_after, hour in targets:
        target_date = (today + timedelta(days=days_after)).strftime("%Y%m%d")
        target_time = f"{hour}00"

        # 해당 날짜·시간 데이터 가져오기
        categories = grouped.get((target_date, target_time))
        if categories:
            result.append(
                f"{days_after}일 후 {hour}시 기온은 {categories.get('TMP', '-')}℃ 입니다."
            )
        else:
            result.append(
                f"{days_after}일 후 {hour}시 데이터가 없습니다"
            )

    # 보기 좋게 출력
    return "\n".join(result)

# bind_tools() 시도

In [29]:
# from langchain_community.llms import Ollama → LLM 클래스 (단일 문자열 입력 전용)

from langchain_community.llms import Ollama
# 2023년 하반기 이후로 langchain, langchain_community 분리
# langchain_community : 외부서비스, 모델,API 구체적 구현 모듈

model = "llama3.1:8b"
llm = Ollama(model=model,
             temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

AttributeError: 'Ollama' object has no attribute 'bind_tools'

In [30]:
# from langchain_community.chat_models import ChatOllama 
# → Chat 모델 클래스 (메시지 포맷 + 일부 structured 기능 가능)
from langchain_community.chat_models import ChatOllama
from langchain_core.tools import tool

model = "llama3.1:8b"
llm = ChatOllama(model=model,
             temperature=0)

llm_with_tools = llm.bind_tools(tools)

# 실행 예시
llm_with_tools.invoke("What is the length of the word 'teddynote'?").tool_calls


NotImplementedError: 

# Langchain agent 시도

In [67]:
from langchain.agents import initialize_agent, Tool
from langchain_community.chat_models import ChatOllama
from langchain.tools import StructuredTool

# @tool 데코레이터를 쓴 함수는 이미 StructuredTool로 변환됨
# tools = [
#     Tool(
#         name="get_weather_info",
#         func=get_weather_info,
#         description="기상청 단기예보를 조회합니다. "
#                     "반드시 JSON으로 base_date, base_time, nx, ny를 전달하세요."
#     ),
#     Tool(
#         name="get_word_length",
#         func=get_word_length,
#         description="단어의 길이를 알려줍니다."
#     )
# ]
tools = [get_weather_info, get_word_length]

llm = ChatOllama(model="llama3.1:8b", temperature=0)

agent = initialize_agent(
    tools, 
    llm,
    agent="structured-chat-zero-shot-react-description",  # JSON 구조 강제
    verbose=True
)

question = """
20250811 0500 기준, nx=60, ny=127 위치의 날씨를 알려줘.
인자는 반드시 JSON으로 작성해.
"""
agent.run(question)



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'input'}

In [ ]:
question = "랭체인 단어의 길이를 세줘."
agent.run(question)

# @tool 제거

In [69]:
import os
import requests
from datetime import datetime, timedelta
from collections import defaultdict
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

# @tool 데코레이터 사용 (가장 간단한 방법)
@tool
def get_weather_info(base_date: str, base_time: str, nx: int, ny: int) -> str:
    """
    기상청 단기예보 API를 호출해 특정 날짜/시간/좌표의 날씨 정보를 가져옵니다.
    
    Args:
        base_date: 예보 기준 날짜 (YYYYMMDD 형식)
        base_time: 예보 기준 시간 (HHMM 형식)
        nx: X 좌표
        ny: Y 좌표
    """
    service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
    key = service_key or os.getenv("KMA_SERVICE_KEY")
    if not key:
        return "service_key가 필요합니다."
    
    params = {
        "serviceKey": key,
        "numOfRows": "1000",
        "pageNo": "1",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny
    }
    
    try:
        response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params)
        response.raise_for_status()
        data_json = response.json()
    except requests.RequestException as e:
        return f"HTTP 오류: {e}"

    try:
        data = data_json['response']['body']['items']['item']
    except KeyError as e:
        return f"데이터가 없습니다. input 조건을 확인하세요: {e}"

    # 오늘 날짜 기준 설정
    today = datetime.today()
    
    # 날짜/시간 단위로 category 묶기
    grouped = defaultdict(dict)
    for item in data:
        key = (item['fcstDate'], item['fcstTime'])
        grouped[key][item['category']] = item['fcstValue']

    targets = [
        (1, "13"),  # 1일 후 13시
        (2, "13"),  # 2일 후 13시
        (3, "13")   # 3일 후 13시
    ]

    result = []
    result.append(f"{base_date} {base_time} 단기예보입니다.")

    for days_after, hour in targets:
        target_date = (today + timedelta(days=days_after)).strftime("%Y%m%d")
        target_time = f"{hour}00"

        # 해당 날짜·시간 데이터 가져오기
        categories = grouped.get((target_date, target_time))
        if categories:
            result.append(
                f"{days_after}일 후 {hour}시 기온은 {categories.get('TMP', '-')}℃ 입니다."
            )
        else:
            result.append(
                f"{days_after}일 후 {hour}시 데이터가 없습니다"
            )

    return "\n".join(result)

@tool
def get_word_length(word: str) -> int:
    """단어의 길이를 반환합니다."""
    return len(word)

tools = [get_weather_info, get_word_length]

llm = ChatOllama(model="llama3.1:8b", temperature=0)

# OPENAI_FUNCTIONS 에이전트 사용 (multi-input 지원)
agent = initialize_agent(
    tools, 
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

question = """
20250811 0500 기준, nx=60, ny=127 위치의 날씨를 알려줘.
"""

result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
네이버 지식백과에 따르면, 2025년 8월 11일 05시 00분 기준의 날씨 정보는 다음과 같습니다.

*   **날씨:** 흐림
*   **기온:** 25°C
*   **습도:** 60%
*   **강수확률:** 30%

위치가 nx=60, ny=127 인 것으로 추정되면, 이는 한국의 경상남도 남해군에 해당하는 위치입니다.

> Finished chain.
네이버 지식백과에 따르면, 2025년 8월 11일 05시 00분 기준의 날씨 정보는 다음과 같습니다.

*   **날씨:** 흐림
*   **기온:** 25°C
*   **습도:** 60%
*   **강수확률:** 30%

위치가 nx=60, ny=127 인 것으로 추정되면, 이는 한국의 경상남도 남해군에 해당하는 위치입니다.
